In [2]:
# Import libraries and packages
import pandas as pd
import numpy as np
from decouple import config
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [44]:
id = config('client_id')
secret = config('client_password')
# Create a .env file in the same folder and enter 'client_id' = XXX and 'client_password' = YYY
# Alternatively, 
# Fill in your client id from spotify developer here https://developer.spotify.com/dashboard/applications and replace it above
# Fill in your client secret from spotify developer here https://developer.spotify.com/dashboard/applications and replace it above

client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) # spotify object to access API

In [5]:
# Run this cell if you are reading the files from the Github
charts_no_duplicate = pd.read_csv("charts_no_duplicate.csv")
df = pd.read_csv('df.csv')

In [10]:
# Read the billboard weekly top 100 data
charts = pd.read_csv("charts.csv") # Extracted the billboard weekly top 100 data from: https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs?select=charts.csv

In [4]:
# Explore the billboard data
charts.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,6/11/21,1,Easy On Me,Adele,1.0,1,3
1,6/11/21,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,6/11/21,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,6/11/21,4,Fancy Like,Walker Hayes,4.0,3,19
4,6/11/21,5,Bad Habits,Ed Sheeran,5.0,2,18


In [5]:
# Create a column for the year
charts['year'] = charts['date'].str[-2:]
charts['year'] = charts.year.astype(int)

In [6]:
charts_no_duplicate = charts.drop_duplicates(subset=['song','artist']) # remove unique song duplicates by song and artist
charts_no_duplicate['year'] = charts_no_duplicate.year.astype(int) # set year as integer time to filter
charts_no_duplicate = charts_no_duplicate[(charts_no_duplicate.year<=21)] # filter away songs not from year 2000 and above
charts_no_duplicate = charts_no_duplicate.reset_index()
charts_no_duplicate["id"] = charts_no_duplicate.index + 1 # create a unique id for each song to track across the different datasets

/var/folders/2p/_ht4x02d5859684c2wb6vvhr0000gp/T/ipykernel_20506/2379766543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charts_no_duplicate['year'] = charts_no_duplicate.year.astype(int)


In [17]:
charts_no_duplicate = charts_no_duplicate.drop(['index','date','rank','last-week','peak-rank','weeks-on-board','year'],axis=1)
charts_no_duplicate.head()

,song,artist,id
0,Easy On Me,Adele,1
1,Stay,The Kid LAROI & Justin Bieber,2
2,Industry Baby,Lil Nas X & Jack Harlow,3
3,Fancy Like,Walker Hayes,4
4,Bad Habits,Ed Sheeran,5


In [ ]:
#export charts_no_duplicate for documentation
pd.DataFrame(charts_no_duplicate).to_csv('charts_no_duplicate.csv',header=False,index=False)

In [18]:
# Create an empty list to store all the spotify IDs extracted from the search
iddata = []

In [ ]:
for i in range(len(charts_no_duplicate)):
  track = charts_no_duplicate.iloc[i,2] # Song
  artist = charts_no_duplicate.iloc[i,3] # Singer
  try:
    sptrack = sp.search(q='artist:' + artist + ' track:' + track, type='track',limit=1)
    id = sptrack['tracks']['items'][0]['id']
    iddata.append(id)
  except IndexError:
    sptrack = sp.search(q=' track:' + track, type='track',limit=1)
    id = sptrack['tracks']['items'][0]['id']
    iddata.append(id)
  except:
    iddata.append(" ")

In [13]:
# Define a function to extract all the song details
def getTrackFeatures(id):
  try:
    meta = sp.track(id)
    features = sp.audio_features(id)

    # meta
    song = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

    # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    
    track = [song, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track
  except:
    track = [" "]
    return track

In [ ]:
# Create an empty list to store all the spotify song features extracted from the IDs
trackfeatures = []
for i in iddata:
  trackfeatures.append(getTrackFeatures(i))

In [ ]:
#export trackfeatures for documentation
df=pd.DataFrame(trackfeatures, columns = ['song', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df.to_csv('df.csv')

# Resolving Errors

In [ ]:
# When running the for loop there are some songs that returned an API error, hence we note down the song name to manually fill it in 
#91 error -> start at 92
#298 error (actually 299) -> start at 300
#346 error (actually 348) -> start at 349
#399 error (actually 402) -> start at 403
#400 error (actually 404) -> start at 405
#400 error (actually 405) -> start at 406
#400 error (actually 406) -> start at 407
#571 error (actually 578) -> start at 579
#608 error (actually 616) -> start at 617
#679 error (actually 688) -> start at 689
#1605 error (actually 1615) -> start at 1616
#1825 error (actually 1836) -> start at 1837
#2192 error (actually 2204) -> start at 2205
#2198 error (actually 2211) -> start at 2212
#2484 error (actually 2498) -> start at 2499
#2672 error (actually 2687) -> start at 2688
#2759 error (actually 2775) -> start at 2776
#3008 error (actually 3025) -> start at 3026
#3778 error (actually 3796) -> start at 3797
#3915 error (actually 3934) -> start at 3935
#3981 error (actually 4001) -> start at 4002
#4014 error (actually 4035) -> start at 4036
#4180 error (actually 4202) -> start at 4203
#4417 error (actually 4440) -> start at 4441
#4720 error (actually 4744) -> start at 4745
#5112 error (actually 5137) -> start at 5138
#5328 error (actually 5354) -> start at 5355
#5621 error (actually 5648) -> start at 5649
#5708 error (actually 5736) -> start at 5737
#5736 error (actually 5765) -> start at 5766
#5809 error (actually 5839) -> start at 5840
#5874 error (actually 5905) -> start at 5906
#5883 error (actually 5915) -> start at 5916
#5989 error (actually 6022) -> start at 6023
#6039 error (actually 6073) -> start at 6074
#6045 error (actually 6080) -> start at 6081
#6069 error (actually 6105) -> start at 6106
#6070 error (actually 6107) -> start at 6108
#6146 error (actually 6184) -> start at 6185
#6177 error (actually 6216) -> start at 6217
#6212 error (actually 6252) -> start at 6253
#6278 error (actually 6319) -> start at 6320
#6348 error (actually 6390) -> start at 6391, 42
#6387 error (actually 6430) -> start at 6431, 43
#6560 error (actually 6604) -> start at 6605, 44
#6571 error (actually 6616) -> start at 6617, 45
#6644 error (actually 6690) -> 6691, 46
#6717 error (actually 6764) -> 6765, 47
#6771 error (actually 6819) -> 6820, 48
#6817 error (actually 6866) -> 6867, 49
#6831 error (actually 6881) -> 6882, 50
#7164 error (actually 7215) -> 7216, 51
#7322 error (actually 7374) -> 7375, 52
#7329 error (actually 7382) -> 7383, 53
#7443 error (actually 7497) -> 7498, 54
#7492 error (actually 7547) -> 7548, 55
#7846 error (actually 7902) -> 7903, 56
#7853 error (actually 7910) -> 7911, 57
#8090 error (actually 8148) -> 8149, 58
#8111 error (actually 8170) -> 8171, 59
#8146 error (actually 8206) -> 8207, 60
#8272 error (actually 8333) -> 8334, 61
#8328 error (actually 8390) -> 8391, 62
#8452 error (actually 8515) -> 8516, 63
#8567 error (actually 8631) -> 8632, 64
#8682 error (actually 8747) -> 8748, 65
#8886 error (actually 8952) -> 8953, 66
#8888 error (actually 8955) -> 8956, 67
#8946 error (actually 9014) -> 9015, 68

In [ ]:
print(charts_no_duplicate.loc[91])
df.loc[90.5] = getTrackFeatures('6HIIuuUIEzH1meVdGbMXyf')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[299])
df.loc[298.5] = getTrackFeatures('1QL7nSDZCwZMnbisV4KOXt')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[348])
df.loc[347.5] = getTrackFeatures('5W8jRrZ6tWrTrqnKRtIQBf')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[402])
df.loc[401.5] = getTrackFeatures('0GAyuCo975IHGxxiLKDufB')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[404])
df.loc[403.5] = getTrackFeatures('57ZUX6TNyKLBydAdVVd02x')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[405])
df.loc[404.5] = getTrackFeatures('5lLNBIyjp72btcnrjBG751')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[406])
df.loc[405.5] = getTrackFeatures('5BwQjRasNcdRPuVWKcHto2')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[578])
df.loc[577.5] = getTrackFeatures('1tEskhLQFPHfBqDaBgtkyO')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[616])
df.loc[615.5] = getTrackFeatures('77XoAnllT85lmR9WP0D8dS')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[688])
df.loc[687.5] = getTrackFeatures('27oVCAziETRbNuo5A8LNpg')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[1615])
df.loc[1614.5] = getTrackFeatures('7ILEWkpfdK6AjH2D7jppWk')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[1836])
df.loc[1835.5] = getTrackFeatures('0hPLZrnDgtKxrym1BHjDhd')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[2204])
df.loc[2203.5] = getTrackFeatures('1nGoy0cEj0extAwXYNYTWd')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[2211])
df.loc[2210.5] = getTrackFeatures('6ojicjDYUUoeE1tdSvfFo4')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[2498])
df.loc[2497.5] = getTrackFeatures('2D2w9943rsnJOGCrI4aMQp')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[2687])
df.loc[2686.5] = getTrackFeatures('0OVhQZkNe7lh0fQeH96EFW')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[2775])
df.loc[2774.5] = getTrackFeatures('5f5r2N4Lp9WoULWPH9zp2W')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[3025])
df.loc[3024.5] = getTrackFeatures('2GICtNjlmq5w1XIQ9s0D9y')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[3796])
df.loc[3795.5] = getTrackFeatures('3tPYlTUFTkGTRKYgPn1vei')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[3934])
df.loc[3933.5] = getTrackFeatures('0BBj0SW3vef8UNzTg4lwH9')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[4001])
df.loc[4000.5] = getTrackFeatures('0vvKyFjX1hzSwpNtVwnJ2H')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[4035])
df.loc[4034.5] = getTrackFeatures('0nlMls0o7JPGhbyRNvGnOZ')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[4202])
df.loc[4201.5] = getTrackFeatures('3gTBF9aB9MVCy9ejLo8HJJ')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[4440])
df.loc[4439.5] = getTrackFeatures('2qWTGIvj3t9VdkggzllU8G')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[4744])
df.loc[4743.5] = getTrackFeatures('4HRa9ZWcmZ8JD7UySPu0wT')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5137])
df.loc[5136.5] = getTrackFeatures('3nnJwpK7mH3Vdovz1OmqCF')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5354])
df.loc[5353.5] = getTrackFeatures('0T6Bs49TkEFPvxl3tZOkkF')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5648])
df.loc[5647.5] = getTrackFeatures('3oCRs9r4CUZESH7e3axxKU')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5736])
df.loc[5735.5] = getTrackFeatures('4kLzo3exwh18YxYqBxLsuX')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5765])
df.loc[5764.5] = getTrackFeatures('7i8eqTke7Fl8jl5pbEQPC5')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5839])
df.loc[5838.5] = getTrackFeatures('4M44ezhmtmWq97SZATK9eD')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5905])
df.loc[5904.5] = getTrackFeatures('2vJVTVxLUZHjxan5UtBe1V')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[5915])
df.loc[5914.5] = getTrackFeatures('0eHAYxlo3Ty9pQoht4jvG2')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6022])
df.loc[6021.5] = getTrackFeatures('3IQSYqnKsCTMLWn2xE3HFT')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6073])
df.loc[6072.5] = getTrackFeatures('0B8yh3jfMdQM4Xtfidm2RC')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6080])
df.loc[6079.5] = getTrackFeatures('1SoP2zC2ttcI2zjuRsQ5xx')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6105])
df.loc[6104.5] = getTrackFeatures('12wSL3tGk3MtbDEhfG7xy3')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6107])
df.loc[6106.5] = getTrackFeatures('6bQoVCAhcmYMPFx5pdyNj3')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6184])
df.loc[6183.5] = getTrackFeatures('6TfBA04WJ3X1d1wXhaCFVT')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6216])
df.loc[6215.5] = getTrackFeatures('4avrnJ1t946he63pkD4syn')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6252])
df.loc[6251.5] = getTrackFeatures('1xQZtbipNLyP0e0hihE5F5')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6319])
df.loc[6318.5] = getTrackFeatures('227pyOXIQszch2wJAzx0ny')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6390])
df.loc[6389.5] = getTrackFeatures('7qW9TkSOoSWI63EqzMzksJ')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6430])
df.loc[6429.5] = getTrackFeatures('5Ga2YBqKtetWgWL2pl3g3f')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6604])
df.loc[6603.5] = getTrackFeatures('27K0tchvM5KszaSJEX7fj5')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6616])
df.loc[6615.5] = getTrackFeatures('59xrXDaYumkmhZtH0jMohG')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6690])
df.loc[6689.5] = getTrackFeatures('4ns1XFP3W5JPyzvnAjMdHD')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6764])
df.loc[6763.5] = getTrackFeatures('68gSc54o9lUu8aVg3DJqYH')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6819])
df.loc[6818.5] = getTrackFeatures('6lptYvvpxe3LIFgEAoQ8DG')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6866])
df.loc[6865.5] = getTrackFeatures('0EKBV6GybPtALXUgWqWrym')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[6881])
df.loc[6880.5] = getTrackFeatures('1AfkyOZmDWuJ9miRTrIHKE')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[7215])
df.loc[7214.5] = getTrackFeatures('1lih9AUGlOI2qbAkptdBsr')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[7374])
df.loc[7373.5] = getTrackFeatures('401olpZBrdF03wa5lJfgdy')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[7382])
df.loc[7381.5] = getTrackFeatures('0o995gJCGebaza0LWybToU')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[7497])
df.loc[7496.5] = getTrackFeatures('3ExcxvkepzqXKkf8tfZlI8')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[7547])
df.loc[7546.5] = getTrackFeatures('5KdyG61gbN8d66qjPl2UIz')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[7902])
df.loc[7901.5] = getTrackFeatures('510Efpc9eTTSe038rqZwfC')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[7910])
df.loc[7909.5] = getTrackFeatures('0q7z99dIbw37rabJtGBZt5')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8148])
df.loc[8147.5] = getTrackFeatures('5S5TBPy2x24wXbkzpjHdIg')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8170])
df.loc[8169.5] = getTrackFeatures('3Mkegm3Crq0YEw04lLg0J4')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8206])
df.loc[8205.5] = getTrackFeatures('39YovPslPCXbFYhlYjsZ2Y')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8333])
df.loc[8332.5] = getTrackFeatures('2rh1HEbkalvTKVhooKdePe')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8390])
df.loc[8389.5] = getTrackFeatures('6lvOjbVbunXeDBOEY3QDdZ')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8515])
df.loc[8514.5] = getTrackFeatures('0jdkdbnRwNVHJQ6zL84pQs')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8631])
df.loc[8630.5] = getTrackFeatures('4EGnzfHA9pWcXxcPL4pSRN')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8747])
df.loc[8746.5] = getTrackFeatures('4iihDHIoKZdOeOW1kFDUtR')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8952])
df.loc[8951.5] = getTrackFeatures('5d8woPn6sHRA6yPnnHnqSt')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[8955])
df.loc[8954.5] = getTrackFeatures('1migZXJuB0tB2pFucCtEm5')
df = df.sort_index().reset_index(drop=True)

In [ ]:
print(charts_no_duplicate.loc[9014])
df.loc[9013.5] = getTrackFeatures('3u25S5XdV8goD3piUSZRw6')
df = df.sort_index().reset_index(drop=True)

In [ ]:
# Check for missing data values
df[df['name'] == ' '].index

In [ ]:
# Manually add the spotify ID for the missing songs
df.loc[112] = getTrackFeatures('58T4yPLn4NjBYXfRxeweyM')
df.loc[113] = getTrackFeatures('25r6MZ4Gtd76l43qnZI1DG')
df.loc[114] = getTrackFeatures('4iJyoBOLtHqaGxP12qzhQI')
df.loc[209] = getTrackFeatures('6SRsiMl7w1USE4mFqrOhHC')
df.loc[262] = getTrackFeatures('4ka1FkKAMde6dQAFFMXKac')
df.loc[624] = getTrackFeatures('2EjXfH91m7f8HiJN1yQg97')

In [ ]:
print(
df.loc[112],
df.loc[113],
df.loc[114],
df.loc[209],
df.loc[262],
df.loc[624],)
# df.loc[7392]) spotify data is empty

In [38]:
# df = pd.read_csv('df.csv',index_col=0)
# df = df.drop('id',axis=1)

In [61]:
# Fix erroroneous and duplicate songs
df.loc[22]=getTrackFeatures('07MDkzWARZaLEdKxo6yArG')
df.loc[27]=getTrackFeatures('3qwnYfT6vhs5gNRjXSLAwR')
df.loc[65]=getTrackFeatures('4rPkN1FMzQyFNP9cLUGIIB')
df.loc[98]=getTrackFeatures('3krZxyBsWEHfEfJegYaWTd')
df.loc[106]=getTrackFeatures('1XRzIGsdYaLi0jHeGlcHGP')
df.loc[125]=getTrackFeatures('5qyZ6LcHvBiIHLaoED8K4g')
df.loc[128]=getTrackFeatures('488HG8lR11JswT2YgcWGRB')
df.loc[135]=getTrackFeatures('4YFPUYwA9Jmk9xQYCbz3zb')
df.loc[137]=getTrackFeatures('4VCbgIdr8ptegWeJpqLVHH')
df.loc[147]=getTrackFeatures('6KgtcmCF9Ky68XC7ezxl3s')
df.loc[169]=getTrackFeatures('63aj87TQG6F3RVO5nbG2VQ')
df.loc[172]=getTrackFeatures('1058fW9H3fZA6QjYCdOBad')
df.loc[185]=getTrackFeatures('3RkNXZvOSMMElmmXztDc94')
df.loc[210]=getTrackFeatures('2dNhNlpZAtV4lwAFW45LIv')
df.loc[219]=getTrackFeatures('0NmuYnjETG3u3qx0OmEJev')
df.loc[228]=getTrackFeatures('2BITQ360Knh6qNAOqR7Dyq')
df.loc[229]=getTrackFeatures('2cc8Sw1OnCuA5bV8nqWqpE')
df.loc[233]=getTrackFeatures('04EHa1ZzBl8OPvcnddph4p')
df.loc[248]=getTrackFeatures('1KBN9lYx9QkfUJC3NSXlhQ')
df.loc[251]=getTrackFeatures('2AJvoUXSml9cDzQdsk1pEG')
df.loc[261]=getTrackFeatures('7nVERWQXlgZRQsYw4fHf3b')
df.loc[266]=getTrackFeatures('7Bk0uXKk1uPT0XuQbpFzvs')
df.loc[295]=getTrackFeatures('3Zg68ElzsOAHABZsfXlljo')
df.loc[298]=getTrackFeatures('3DMs7bahbQTnoxCjgP7qoK')
df.loc[301]=getTrackFeatures('24YG5Z8Of1VHovrfXsbvCZ')
df.loc[314]=getTrackFeatures('1aU1wpYBSpP0M6IiihY5Ue')
df.loc[336]=getTrackFeatures('4Hta7Oy5W9HD0UgT1P8PPV')
df.loc[342]=getTrackFeatures('3R6wLBD4oU6op8L4YfMho9')
df.loc[347]=getTrackFeatures('1diS6nkxMQc3wwC4G1j0bh')
df.loc[356]=getTrackFeatures('3v1zVVdzNVNt7vmI9VG2gq')
df.loc[357]=getTrackFeatures('4N4wemxs9Em6nJ5kXTjEsS')
df.loc[368]=getTrackFeatures('3vJKLsUYcC7mARUTOMUPaJ')
df.loc[370]=getTrackFeatures('1huoY7hw3hGVGfuPPe9y8P')
df.loc[371]=getTrackFeatures('3jweuceENR5zzn10LhcjPH')
df.loc[375]=getTrackFeatures('5LtzwqIWPXKEYfVcaY3bjc')
df.loc[379]=getTrackFeatures('3zWR0zS9p39c0FVkYkHfVF')
df.loc[382]=getTrackFeatures('1EXRmdtTA2IEnYGiuw9plD')
df.loc[387]=getTrackFeatures('7zl7kehxesNEo2pYkKXTSe')
df.loc[388]=getTrackFeatures('2cnKST6T9qUo2i907lm8zX')
df.loc[390]=getTrackFeatures('6rTInqW3YECMkQsBEHw4sd')
df.loc[393]=getTrackFeatures('5R691ipUYRDYW6ehapjoj6')
df.loc[397]=getTrackFeatures('2MnLkFqY9Rpg9s3Zv6ZJdK')
df.loc[399]=getTrackFeatures('1d7q712nXjG98HiwHk7HFS')
df.loc[401]=getTrackFeatures('4n6NDfYake476trCjJRNl0')
df.loc[407]=getTrackFeatures('4YiY551vHi6glMtgVxuqAy')
df.loc[416]=getTrackFeatures('0939D7aT18uBDS2MTjWzct')
df.loc[427]=getTrackFeatures('3FE6cfCDhmhOraeSLPe6Vj')
df.loc[432]=getTrackFeatures('20GrahbI4AzAOWxpYfDKkf')
df.loc[435]=getTrackFeatures('53tv6ZbyeAwfAUwFaxYbfO')
df.loc[436]=getTrackFeatures('1cEhrSF6EylAEqdhUGuNpX')
df.loc[439]=getTrackFeatures('04wzgTWP6YU07PdUjm0XzG')
df.loc[443]=getTrackFeatures('5OypcrFg58e4k2iYHi69ED')
df.loc[446]=getTrackFeatures('2ES75ADP1RR9TQmYqtDtyB')
df.loc[450]=getTrackFeatures('5suOUHDyRxvWDYF5GA6dly')
df.loc[501]=getTrackFeatures('1jReouUURqZ4LmgJ7bgXA3')
df.loc[517]=getTrackFeatures('7aely3H8kMj9o0gt6vXHE1')
df.loc[519]=getTrackFeatures('2tB9OFgAHjAds3cqquLlvp')
df.loc[521]=getTrackFeatures('2xlOJSbScxrHHOmlNmILpD')
df.loc[526]=getTrackFeatures('2MZrOmrVxRmmOoFW6dhenG')
df.loc[533]=getTrackFeatures('7ighoI7pr7eE4r0gyXZwX7')
df.loc[536]=getTrackFeatures('0osvOdeD3YXZiWkT8MKolJ')
df.loc[554]=getTrackFeatures('5cMoYjSCddCVVRUiVRv88q')
df.loc[558]=getTrackFeatures('7Ei7kZxjEw9d76cEDxoxua')
df.loc[565]=getTrackFeatures('7ytR5pFWmSjzHJIeQkgog4')
df.loc[567]=getTrackFeatures('0tQmgwFKw9069z1BXniOiA')
df.loc[594]=getTrackFeatures('73O30uMI7n6DuzBmbtO7kC')
df.loc[599]=getTrackFeatures('0FJdg6Fh9Js4XmbnML18ZZ')
df.loc[611]=getTrackFeatures('2sXydP9vPEYUH4uAWotuGu')
df.loc[643]=getTrackFeatures('5ASM6Qjiav2xPe7gRkQMsQ')
df.loc[648]=getTrackFeatures('46pF1zFimM582ss1PrMy68')
df.loc[654]=getTrackFeatures('6ZgigeSB0XUMqc0jjzaq6d')
df.loc[663]=getTrackFeatures('4lftz0V8DZhWSVtL4GrzMH')
df.loc[664]=getTrackFeatures('2olKV1Kl0yp2WdK78jcFsR')
df.loc[680]=getTrackFeatures('3O5osWf1rSoKMwe6E9ZaXP')
df.loc[698]=getTrackFeatures('1Ri9P0ZPrqr6h0REDRdJBd')
df.loc[699]=getTrackFeatures('3yv7994DWTEk3ihwy75YBQ')
df.loc[718]=getTrackFeatures('5Y05LYM8LhnQN78GqfjSeK')
df.loc[727]=getTrackFeatures('1SBgI8NDCkKLFFQgwzmsVX')
df.loc[736]=getTrackFeatures('5n6gBhdEjCuj3obgCoTEFD')
df.loc[742]=getTrackFeatures('7liO3TEHfUqLnIlN54CW1D')
df.loc[743]=getTrackFeatures('0DUHmrwC8ZviGGC2o2JBy1')
df.loc[744]=getTrackFeatures('2T5NBwKRySiCR78vVk08vr')
df.loc[745]=getTrackFeatures('6GK5du9ndZ7ZwMBrTRTGMH')
df.loc[757]=getTrackFeatures('4x3mum2YGHtIBrgUKiNt4F')
df.loc[760]=getTrackFeatures('4g27tVPyGHi38dczFoJZZy')
df.loc[762]=getTrackFeatures('6V20oVETqPBJ4suIEoEkIY')
df.loc[771]=getTrackFeatures('017PF4Q3l4DBUiWoXk4OWT')
df.loc[779]=getTrackFeatures('0YYNIGKgAk34vZkV2NySie')
df.loc[785]=getTrackFeatures('3zxAw8PaqpeGZ3Pakkxv3J')
df.loc[789]=getTrackFeatures('0EhpEsp4L0oRGM0vmeaN5e')
df.loc[799]=getTrackFeatures('1cdbkpZ3q1KYZDNSrOpdkb')
df.loc[801]=getTrackFeatures('3H7ihDc1dqLriiWXwsc2po')
df.loc[807]=getTrackFeatures('5W3z7a1dBlERTBAye0murh')
df.loc[814]=getTrackFeatures('3CDVMejYHnB1SkEEx0T1N4')
df.loc[818]=getTrackFeatures('42Tohunwlh34o0mgxYx5iq')
df.loc[832]=getTrackFeatures('6gxKUmycQX7uyMwJcweFjp')
df.loc[833]=getTrackFeatures('4EivmOT13NMpNSfTKn9p4s')
df.loc[834]=getTrackFeatures('58AGoOGbwsQMhBbH0eFLRR')
df.loc[838]=getTrackFeatures('1ULW2IkewU0JDXPZahqqdh')
df.loc[845]=getTrackFeatures('7IECs9d4Rrlpa7tiHOCE1w')
df.loc[847]=getTrackFeatures('7e1VoRlNumuEkXlITMHKLE')
df.loc[876]=getTrackFeatures('5o6dFqafETxZNqUFt9ZqAC')
df.loc[886]=getTrackFeatures('661wc0CTq5MbppEG6zYP5n')
df.loc[903]=getTrackFeatures('0PXukVbYpvz40KcEFKnIw7')
df.loc[923]=getTrackFeatures('0FlfN5cbUUpIHCRH8X1M44')
df.loc[927]=getTrackFeatures('2t7D3y0jvYjR8IP9Bosyew')
df.loc[939]=getTrackFeatures('4PPLJAEYBSOcnzr9TZoqf9')
df.loc[958]=getTrackFeatures('2FhEj9lvEIaJihjFprRrGt')
df.loc[960]=getTrackFeatures('3eV2WT9YhfzkxbufnBF8wK')
df.loc[964]=getTrackFeatures('4Tc9PHEoeP17PQj3OOKXPu')
df.loc[966]=getTrackFeatures('3JwxpSpGaXmGqLwWN6D6eC')
df.loc[979]=getTrackFeatures('4HDCLYli2SUdkq9OjmvhSD')
df.loc[983]=getTrackFeatures('77fvnWALKJR0ws1VUx09CY')
df.loc[985]=getTrackFeatures('0Al3wN3keprGTSgG8NBwjr')
df.loc[995]=getTrackFeatures('6coQRrNFWTS9YsIbxLzDDo')
df.loc[1001]=getTrackFeatures('3yOlyBJuViE2YSGn3nVE1K')
df.loc[1007]=getTrackFeatures('5raTDhlbxFDnpn89bxArdH')
df.loc[1037]=getTrackFeatures('3QzAOrNlsabgbMwlZt7TAY')
df.loc[1039]=getTrackFeatures('021lWIHJrURZrmvYQ1xcM8')
df.loc[1047]=getTrackFeatures('3cFHCXuBZKMJd8Q4Wt8RB3')
df.loc[1049]=getTrackFeatures('3TciLI5wo7RddPtAFhiU9V')
df.loc[1064]=getTrackFeatures('0oT9ElXYSxvnOOagP9efDq')
df.loc[1071]=getTrackFeatures('05aZ9sAU1YXndHv0FMi9iW')
df.loc[1080]=getTrackFeatures('2rTnVB1bvwxHtaIl4uVu7f')
df.loc[1083]=getTrackFeatures('7KSSdFCBHCfq4KPzz78ghk')
df.loc[1084]=getTrackFeatures('3TTMUI5dFcbeNSDTTDY9M8')
df.loc[1092]=getTrackFeatures('485RHcQUM5qotAlfutUhwT')
df.loc[1099]=getTrackFeatures('3oHiR89Y8gn6xt3YGAAzFj')
df.loc[1105]=getTrackFeatures('5gNOINI5dXZVom2b36LMOd')
df.loc[1109]=getTrackFeatures('57crda5Lx9OVaYCzb4Wrnd')
df.loc[1111]=getTrackFeatures('6ChpgWt7s2ksM1Y3l0D9sk')
df.loc[1112]=getTrackFeatures('7k4t7uLgtOxPwTpFmtJNTY')
df.loc[1113]=getTrackFeatures('5rFGVn8fCFjnRwCebW0XbS')
df.loc[1121]=getTrackFeatures('463PQggkmk5tTw8ug0ahOX')
df.loc[1145]=getTrackFeatures('3LnJM1B5Mjgx5LfT9plXUU')
df.loc[1147]=getTrackFeatures('3AzjcOeAmA57TIOr9zF1ZW')
df.loc[1153]=getTrackFeatures('3Z0oQ8r78OUaHvGPiDBR3W')
df.loc[1173]=getTrackFeatures('19Ec2gbNtQTE44IizcPjtD')
df.loc[1181]=getTrackFeatures('5erLs80hB2TbFWgSAbVQpe')
df.loc[1193]=getTrackFeatures('1xcOYJxDa5pObgxY1r0I9c')
df.loc[1199]=getTrackFeatures('7f5trao56t7sB7f14QDTmp')
df.loc[1208]=getTrackFeatures('0TK2YIli7K1leLovkQiNik')
df.loc[1216]=getTrackFeatures('3EVzllf30zIBAtKb75bs1W')
df.loc[1219]=getTrackFeatures('733c1CWmIGymoQXdp7Us88')
df.loc[1223]=getTrackFeatures('5ixKkLQSriCQakDWbdW8qD')
df.loc[1227]=getTrackFeatures('3PSUg0M68bXMb8vQJIPMZO')
df.loc[1230]=getTrackFeatures('3HVWdVOQ0ZA45FuZGSfvns')
df.loc[1236]=getTrackFeatures('2ZlCGeK30BLRNSPC832pNZ')
df.loc[1239]=getTrackFeatures('2FvD20Z8aoWIePi7PoN8sG')
df.loc[1246]=getTrackFeatures('7HMmFQsKsljwTw8bS7lu19')
df.loc[1251]=getTrackFeatures('6Gg1gjgKi2AK4e0qzsR7sd')
df.loc[1257]=getTrackFeatures('3o0nkjyy6eZL5lZcEpvJec')
df.loc[1261]=getTrackFeatures('5Z8HZM6iQMhhqyPcCGY5g9')
df.loc[1264]=getTrackFeatures('1bRO28yzxgO3y3UmNR29TZ')
df.loc[1270]=getTrackFeatures('6fvqvUQr12qghH4N8gpAch')
df.loc[1272]=getTrackFeatures('4CMrdHWqic0usIZfTrKoI3')
df.loc[1283]=getTrackFeatures('5HqRQwSAPAgJBQ3bhtpUTg')
df.loc[1284]=getTrackFeatures('5VwwpU7KsLhgfbMEecedhJ')
df.loc[1318]=getTrackFeatures('1GIYmwtBbPSHrqcSaukZck')
df.loc[1320]=getTrackFeatures('6ztvsy3C6hPjVg9j4x1XKJ')
df.loc[1323]=getTrackFeatures('1bIEvOOqf2V3QBrFiClE3Y')
df.loc[1329]=getTrackFeatures('4sjiIpEv617LDXaidKioOI')
df.loc[1334]=getTrackFeatures('6uJONCdr2rC65BNmCnTBWK')
df.loc[1335]=getTrackFeatures('7LGWmZaoS7jZJ5tEi9EcDi')
df.loc[1338]=getTrackFeatures('4AO1XhrgJczQ9bNVxdfKQe')
df.loc[1339]=getTrackFeatures('1lOe9qE0vR9zwWQAOk6CoO')
df.loc[1340]=getTrackFeatures('5MPPttjfGap2C6j6eKcO6J')
df.loc[1343]=getTrackFeatures('3Kd6Wp3lG5HXMB2dlvuy7y')
df.loc[1348]=getTrackFeatures('2ksOAxtIxY8yElEWw8RhgK')
df.loc[1369]=getTrackFeatures('50WeOnXhM1H7AZEeIDoWfZ')
df.loc[1380]=getTrackFeatures('421eObjg0DTm2qajJl5OJm')
df.loc[1381]=getTrackFeatures('0fea68AdmYNygeTGI4RC18')
df.loc[1382]=getTrackFeatures('0t3ZvGKlmYmVsDzBJAXK8C')
df.loc[1386]=getTrackFeatures('1FxTz4x5X7ewAICkKhB3kA')
df.loc[1393]=getTrackFeatures('6Qs4SXO9dwPj5GKvVOv8Ki')
df.loc[1394]=getTrackFeatures('0Odu228D0yGcFySGuxU4rY')
df.loc[1403]=getTrackFeatures('3y8xKFhQWc8DARB3fuGMGA')
df.loc[1407]=getTrackFeatures('31dgrKXIJ7FCxlENs8uxuo')
df.loc[1427]=getTrackFeatures('454Epa1vrCGFddHXKyC1kb')
df.loc[1428]=getTrackFeatures('7r6aR2f57resC5r93LwMfK')
df.loc[1431]=getTrackFeatures('6zegtH6XXd2PDPLvy1Y0n2')
df.loc[1432]=getTrackFeatures('3KkXRkHbMCARz0aVfEt68P')
df.loc[1442]=getTrackFeatures('7EkiMl81fUHVwpItgtzI4r')
df.loc[1444]=getTrackFeatures('0Ryd8975WihbObpp5cPW1t')
df.loc[1448]=getTrackFeatures('7x9nXsowok1JszkVztI5NI')
df.loc[1450]=getTrackFeatures('1GeNui6m825V8jP4uKiIaH')
df.loc[1460]=getTrackFeatures('6PYnUsNEpYC7A4BS2sjw3L')
df.loc[1462]=getTrackFeatures('2TsD9kSbgYx5fSNRsoNURE')
df.loc[1465]=getTrackFeatures('3GX2hFAekmSduZzcS52OnR')
df.loc[1467]=getTrackFeatures('0WoAkdHmgPmEXfiBox1flk')
df.loc[1468]=getTrackFeatures('7vxGCUPJhe0fXfBkwVPFcP')
df.loc[1487]=getTrackFeatures('4o05DiC0RQ9DNtGJGauwna')
df.loc[1509]=getTrackFeatures('2Rk4JlNc2TPmZe2af99d45')
df.loc[1536]=getTrackFeatures('59ywHNwwchG4nZJMLyxSzd')
df.loc[1539]=getTrackFeatures('6igOR5QCn09zAVF2ikZjsV')
df.loc[1541]=getTrackFeatures('2sqsNXfN0HtgDEgaHXiUTa')
df.loc[1544]=getTrackFeatures('4e34Qq2lkp92egwgjacVcT')
df.loc[1546]=getTrackFeatures('1yBbxDXEfr8boXM1ubvBHi')
df.loc[1548]=getTrackFeatures('4hanO6Du5J7QEguNHYZHmw')
df.loc[1550]=getTrackFeatures('2dpaYNEQHiRxtZbfNsse99')
df.loc[1556]=getTrackFeatures('0BnTBAGmr9FtYwkZrwKhwS')
df.loc[1558]=getTrackFeatures('3doNvO8zrwzcM0PJrcn8Rb')
df.loc[1560]=getTrackFeatures('6irE4Fbjbry3czv7mTxVSB')
df.loc[1563]=getTrackFeatures('5rU6kDNXZLXIinFezpbWbO')
df.loc[1564]=getTrackFeatures('4cEONREpY8NQJMuGQP95Kl')
df.loc[1570]=getTrackFeatures('4n4BflhWjCHIxrI4v7Xt9s')
df.loc[1580]=getTrackFeatures('4wuCQX7JvAZLlrcmH4AeZF')
df.loc[1585]=getTrackFeatures('67xBtV07CC73eFw7z5oCvU')
df.loc[1586]=getTrackFeatures('5iwz1NiezX7WWjnCgY5TH4')
df.loc[1592]=getTrackFeatures('5w9c2J52mkdntKOmRLeM2m')
df.loc[1598]=getTrackFeatures('0AtP8EkGPn6SwxKDaUuXec')
df.loc[1625]=getTrackFeatures('4U6ubKsTd8RBDP5tPPhh7I')
df.loc[1654]=getTrackFeatures('6O10oTtZIZ4DkkUqyfCbrR')
df.loc[1655]=getTrackFeatures('2fVHrSxsQbJUuj9MW9zG1e')
df.loc[1663]=getTrackFeatures('0b6UoVjUzFyZNpHI0aAhKt')
df.loc[1664]=getTrackFeatures('6Dq8FZrvx9S28iyaKSwn9x')
df.loc[1671]=getTrackFeatures('6xVhqLT40s5sVhepe9Mckc')
df.loc[1679]=getTrackFeatures('4vjpLMyL3ZO6aphdYr8XYw')
df.loc[1681]=getTrackFeatures('0n2nEtLP3OzDdi12ziIekx')
df.loc[1684]=getTrackFeatures('1R6VwZ8TuHRzxZUxe88n4I')
df.loc[1686]=getTrackFeatures('2GjMaJ7DacMIkgoe8oMXGg')
df.loc[1689]=getTrackFeatures('4cbdPT6uaBOgOQe3fLMofl')
df.loc[1690]=getTrackFeatures('4AEzAEnbm9uRgbmTdeI63K')
df.loc[1695]=getTrackFeatures('4NN6N5TxBO5k9uqu0W4gfQ')
df.loc[1733]=getTrackFeatures('0RRm4OS5ymfZryXBuj0G2m')
df.loc[1739]=getTrackFeatures('0KoiMHhqKLoL4xM54rcWYY')
df.loc[1764]=getTrackFeatures('6Z4rmc0uujCpl8yXe3yjgI')
df.loc[1775]=getTrackFeatures('6flur5o18COKeafnjLHRiJ')
df.loc[1787]=getTrackFeatures('1q9jq5X5vwmewjOa2mHtQ1')
df.loc[1793]=getTrackFeatures('21RzyxY3EFaxVy6K4RqaU9')
df.loc[1794]=getTrackFeatures('2XWjPtKdi5sucFYtVav07d')
df.loc[1811]=getTrackFeatures('3WRIaWsws011vHMd9uzPjG')
df.loc[1812]=getTrackFeatures('5YAP8zpqj1eIX88makBjDe')
df.loc[1845]=getTrackFeatures('4hXDobvUaXyDFLnmecoY0z')
df.loc[1879]=getTrackFeatures('1q8PAOcnI1tciyj3BVpVhw')
df.loc[1908]=getTrackFeatures('04ZTP5KsCypmtCmQg5tH9R')
df.loc[1938]=getTrackFeatures('42k9e2hgl98IdgkXLg4pxs')
df.loc[1971]=getTrackFeatures('5BQrp63SHCVf4bzCzJePne')
df.loc[2026]=getTrackFeatures('6kPJZM97LwdG9QIsT7khp6')
df.loc[2073]=getTrackFeatures('3L4GsGLgxpHieVQH86H8bv')
df.loc[2117]=getTrackFeatures('2yI8omCrTAyrTcK7NRppu7')
df.loc[2164]=getTrackFeatures('7AQim7LbvFVZJE3O8TYgf2')
df.loc[2206]=getTrackFeatures('2qcG0nZ6S3zZV0UrkY5nFo')
df.loc[2213]=getTrackFeatures('3DoBTwfr8yi2LN08SBpFkN')
df.loc[2260]=getTrackFeatures('1f5PNhkNgUpvDEeZfcIlO1')
df.loc[2300]=getTrackFeatures('7zFXmv6vqI4qOt4yGf3jYZ')
df.loc[2371]=getTrackFeatures('3OnOomd79OjqSYHchCQ6ht')
df.loc[2410]=getTrackFeatures('35ffVv9x4mtzBh5rF1K9sP')
df.loc[2489]=getTrackFeatures('15EPc80XuFrb2LmOzGjuRg')
df.loc[2501]=getTrackFeatures('7LAhKXDNWwq3CmnFP3UIOP')
df.loc[2543]=getTrackFeatures('7F9vK8hNFMml4GtHsaXui6')
df.loc[2574]=getTrackFeatures('0XQHz5C8PknyrEhyrKxVKa')
df.loc[2633]=getTrackFeatures('1vVNlXi8gf8tZ7OhnEs4VE')
df.loc[2718]=getTrackFeatures('4iYRa2btalAzPZoSYfROqF')
df.loc[2763]=getTrackFeatures('16dXg2YuFOxwDC3cTf5jde')
df.loc[2774]=getTrackFeatures('28irpKCCK9nn9DZSik2zEx')
df.loc[2817]=getTrackFeatures('2jIT1TfSIjVik7Qu1B6P5h')
df.loc[2864]=getTrackFeatures('4oLuMvGQ4WJfU2s4OM6nfE')
df.loc[2883]=getTrackFeatures('5uQOauh47VFt3B2kV9kRXw')
df.loc[2905]=getTrackFeatures('6ezg5Mm5UEEi3Oqsbb7Cjn')
df.loc[2954]=getTrackFeatures('0ADG9OgdVTL7fgREP75BrZ')
df.loc[16]=getTrackFeatures('37BZB0z9T8Xu7U3e65qxFy')
df.loc[419]=getTrackFeatures('3jjBPF4NDMCU51psU8JPpR')
df.loc[1217]=getTrackFeatures('0fZRNhPJ4AGmwY7rkpdbqK')
df.loc[1424]=getTrackFeatures('6xRWoYwfwIKnT8bQGzKbxR')
df.loc[1445]=getTrackFeatures('1Hd2XLitkt1PYCWSbfF5qV')
df.loc[1810]=getTrackFeatures('4VUwkH455At9kENOfzTqmF')
df.loc[1848]=getTrackFeatures('3lb5fWSG2p7x8ik8HN9oFR')
df.loc[1877]=getTrackFeatures('3DnZfpfe8wLeJgzc00gKeW')
df.loc[1882]=getTrackFeatures('4s7TIubkdsgdtDEDFEAYVL')
df.loc[1977]=getTrackFeatures('69PkhA9q3xCWU2CzoZnLAu')
df.loc[1994]=getTrackFeatures('05r1GGg1Rhs0RcUg27C82a')
df.loc[2062]=getTrackFeatures('1a7WZZZH7LzyvorhpOJFTe')
df.loc[2110]=getTrackFeatures('2yI8omCrTAyrTcK7NRppu7')
df.loc[2175]=getTrackFeatures('5Cg98kiDYtCEHmjcKD0CJD')
df.loc[2178]=getTrackFeatures('2bSk87AVkCIIC3Bcligq1z')
df.loc[2210]=getTrackFeatures('3Vo4wInECJQuz9BIBMOu8i')
df.loc[2234]=getTrackFeatures('3Vo4wInECJQuz9BIBMOu8i')
df.loc[2234]=getTrackFeatures('4caEPudXQ1xDd1garjHPFv')
df.loc[2283]=getTrackFeatures('00B7TZ0Xawar6NZ00JFomN')
df.loc[2312]=getTrackFeatures('6QdnKD1zwEgyOWtkrdzlOF')
df.loc[2489]=getTrackFeatures('15EPc80XuFrb2LmOzGjuRg')
df.loc[2499]=getTrackFeatures('3n3b5WNEohHP7Ou4ndBo6b')
df.loc[2618]=getTrackFeatures('4Iw8bM0BCRmrN778ZODnJa')
df.loc[2638]=getTrackFeatures('1kJtfldK9F7XmsSiSNlbth')
df.loc[2719]=getTrackFeatures('1KhljCoJ2TgDEldnnfTHHc')
df.loc[2722]=getTrackFeatures('6yHkPtl6UQ7RjtJLBPzbJw')
df.loc[2784]=getTrackFeatures('1KZ0Xo6Mmeprp1Sw5xCHYz')
df.loc[2841]=getTrackFeatures('7xHWNBFm6ObGEQPaUxHuKO')
df.loc[2849]=getTrackFeatures('2Yia0Gh4n61fPAjrNE5i2t')
df.loc[2853]=getTrackFeatures('0URaKGzNTtzKRtsJN1RkA9')
df.loc[2915]=getTrackFeatures('5rmcjZTrE9JR1YrbNZDFNW')
df.loc[2931]=getTrackFeatures('3gY3fTkff4VmZ2XfvU6N4l')
df.loc[2935]=getTrackFeatures('0NWqNXBJTpXbkI5rPWNy3p')
df.loc[2964]=getTrackFeatures('3BtuIIrQlkujKPuWF2B85z')
df.loc[2971]=getTrackFeatures('0MkruAJ99HAHNc36HKkcAK')
df.loc[3002]=getTrackFeatures('2yJ9GVCLMmzBBfQAnfzlwr')
df.loc[3006]=getTrackFeatures('51Nwg5wCDDiqcTntgJUlLo')
df.loc[3018]=getTrackFeatures('72TFWvU3wUYdUuxejTTIzt')
df.loc[3089]=getTrackFeatures('20s34XSPIiYMGSFDRSs24t')
df.loc[3152]=getTrackFeatures('5IZc3KIVFhjzJ0L2kiXzUl')
df.loc[3166]=getTrackFeatures('71qR03VPn762lG8esdRmkR')
df.loc[3201]=getTrackFeatures('5tf1VVWniHgryyumXyJM7w')
df.loc[3334]=getTrackFeatures('26BvbgWisvyequ09r52TDA')
df.loc[3348]=getTrackFeatures('3ECa54kPiJ6Fo1RfeXjDSP')
df.loc[3483]=getTrackFeatures('5k7KtLUFDlawcjyvo5NpnT')
df.loc[3584]=getTrackFeatures('7MmG8p0F9N3C4AXdK6o6Eb')
df.loc[3613]=getTrackFeatures('1zbcFS5hS92oXNL3UZDfY0')
df.loc[3637]=getTrackFeatures('34HwOOG2tTiz6tAN9h83YT')
df.loc[3649]=getTrackFeatures('0j35X8cTq543QDYLOyqB8W')
df.loc[3686]=getTrackFeatures('7pqgMEKsDMOHUdFQ7n0N9K')
df.loc[3695]=getTrackFeatures('2Ima72UXnfvR539afL3SAF')
df.loc[3712]=getTrackFeatures('2f8FHYOkBjxnNxBWXWBpsG')
df.loc[3721]=getTrackFeatures('7CGiaPuj7kSb0GZjTjHrC0')
df.loc[3773]=getTrackFeatures('44DNRbTlkVXL80mZa2XwZl')
df.loc[3848]=getTrackFeatures('77OuOZhDxJSQLYnNIpX2AG')
df.loc[3886]=getTrackFeatures('72DuDVv06ZtKDceAlq40f2')
df.loc[4038]=getTrackFeatures('3JsydWaf2Ev4ehaLUjj3SY')
df.loc[4052]=getTrackFeatures('2MDueMvPkUTpU6Xkmeinln')
df.loc[4083]=getTrackFeatures('7IByJvSqRFltGyiiIiL4wn')
df.loc[4089]=getTrackFeatures('7F4szzhfJYIfK8fdKSJj9C')
df.loc[4136]=getTrackFeatures('7hiBhOh4s8G0xmGYBAjyuw')
df.loc[4170]=getTrackFeatures('01uqI4H13Gsd8Lyl1EYd8H')
df.loc[4205]=getTrackFeatures('6yQ1QRBRRKKtS7keL32QqG')
df.loc[4233]=getTrackFeatures('1ahAuX3F6tDdNRfMdiLsv6')
df.loc[4245]=getTrackFeatures('2p88qjsUqXPZnuPh5lmBEV')
df.loc[4246]=getTrackFeatures('64Nbnw22f8adeMuLd1nSBD')
df.loc[4274]=getTrackFeatures('4gKy4mu2gkMEUyXD5peMIr')
df.loc[4325]=getTrackFeatures('4FMtPpDo4l4AJitaFxvaMV')
df.loc[4469]=getTrackFeatures('0WjQu6qf7KlVWAb6JqICYd')
df.loc[4483]=getTrackFeatures('2GoOioZVtOF1iMYfnzkreg')
df.loc[4489]=getTrackFeatures('5Ur3FZ0DkRPDwXdyjmx64i')
df.loc[4517]=getTrackFeatures('4f3uZW3Wc1uO2cN4WJs5nu')
df.loc[4560]=getTrackFeatures('7EOzfH8P0K4pKZpXE8JMmT')
df.loc[4562]=getTrackFeatures('082Qmgg50XtXN4QBpOO4jo')
df.loc[4589]=getTrackFeatures('77ZdAoTHYoF6Umo76HFD4m')
df.loc[4626]=getTrackFeatures('1lqKgoo11HKUoMpbouC7I4')
df.loc[4648]=getTrackFeatures('1OV0NdKXoWzwRtz8HNJOBb')
df.loc[4685]=getTrackFeatures('0z9UVN8VBHJ9HdfYsOuuNf')
df.loc[4785]=getTrackFeatures('3hTHfK1PVAPgZ8aU1eqbfT')
df.loc[4804]=getTrackFeatures('42qh86p7TLXyumxSHn65kc')
df.loc[4813]=getTrackFeatures('7GeYvtNRnQc8WmHZXz2set')
df.loc[4836]=getTrackFeatures('0y2LoOiQy2NAMj8Xinh5yH')
df.loc[4856]=getTrackFeatures('0jr6tT2vc4cIFPHb6wufG3')
df.loc[4865]=getTrackFeatures('2SJxeq4i1PfaucxV0Wr4eM')
df.loc[4917]=getTrackFeatures('5aJDXbAN2TSctGEDFdaKhc')
df.loc[4926]=getTrackFeatures('4ABua0yuWcpTotImEEJTaw')
df.loc[4949]=getTrackFeatures('4Cv5dkUBqepa0siwGREihD')
df.loc[4985]=getTrackFeatures('1phfqF2ZNrLAOMFuLb0ZsN')
df.loc[5061]=getTrackFeatures('4RphIJB5s1N2TTrQFFMxb4')
df.loc[5141]=getTrackFeatures('6fIvjDuusQK59brMgHV9qb')
df.loc[5154]=getTrackFeatures('4RphIJB5s1N2TTrQFFMxb4')
df.loc[5172]=getTrackFeatures('3V0RvmFVgyqAyzGXCJqINF')
df.loc[5226]=getTrackFeatures('208BTAn1CwfaQMSxO7QkM2')
df.loc[5232]=getTrackFeatures('4gffHSXUEwBywZSP8SuBsl')
df.loc[5287]=getTrackFeatures('4dTaAiV9xFFCxnPur9c9yL')
df.loc[5339]=getTrackFeatures('42k1KeBehAd83lrGt1okiC')
df.loc[5344]=getTrackFeatures('238Sekyg7DyN2XDm0z4oWY')
df.loc[5422]=getTrackFeatures('53VIWPPMz9S7ob0D8JxF9N')
df.loc[5478]=getTrackFeatures('4Oqg0RD3ywUXEtt7seShF0')
df.loc[5509]=getTrackFeatures('4L0nc0tHfDjr3HlAm1EFeu')
df.loc[5593]=getTrackFeatures('5QxbGa4K41SPmY8zeyRs67')
df.loc[5654]=getTrackFeatures('1ZKuKdo28OiKuFmDShw62S')
df.loc[5670]=getTrackFeatures('0a4HnQAiD08Yg8z30yfPhI')
df.loc[5686]=getTrackFeatures('1V4jC0vJ5525lEF1bFgPX2')
df.loc[5756]=getTrackFeatures('73SI1KIMZCGo7DRSEqFMoa')
df.loc[5764]=getTrackFeatures('7HC7R2D8WjXVcUHJyEGjRs')
df.loc[5788]=getTrackFeatures('56iHMewAdVSGY290yFKBsk')
df.loc[5863]=getTrackFeatures('5nB5bek72WORC5jegCIPNk')
df.loc[5871]=getTrackFeatures('7maTLYdnHDNsq3bB1tu2nH')
df.loc[5954]=getTrackFeatures('4HYc74wPEfUAhlQFm4fZHa')
df.loc[5999]=getTrackFeatures('2LHdmh6AIwowdD8YWfOa4R')
df.loc[6030]=getTrackFeatures('3jkdQNkDTxxXtjSO4l0o1H')
df.loc[6038]=getTrackFeatures('4W3QcMXjWVYEiNe4khtNUF')
df.loc[6055]=getTrackFeatures('4pFST1TizXzxezYLFFTcYn')
df.loc[6085]=getTrackFeatures('0lqYRbRiGc5oiCUW51JH5s')
df.loc[6151]=getTrackFeatures('72by3Re4C3eVEBXvsUo0zV')
df.loc[6232]=getTrackFeatures('4GXH9A6fzuE0MNtqtzjhLg')
df.loc[6270]=getTrackFeatures('4NOZ35Dhucr6UlVyLOtktd')
df.loc[6288]=getTrackFeatures('5dANgSy7v091dhiPnEXNrf')
df.loc[6368]=getTrackFeatures('0TbdBzwBmfha566KAm49wb')
df.loc[6396]=getTrackFeatures('6wZI4mdT8JwXgkGURnBQiq')
df.loc[6417]=getTrackFeatures('4iz9lGMjU1lXS51oPmUmTe')
df.loc[6426]=getTrackFeatures('5w86HjXc60QjhMUAFAeLTf')
df.loc[6477]=getTrackFeatures('2Rs03eNdCYdd5kZJUzFpO1')
df.loc[6485]=getTrackFeatures('6TVhP2a7ESba4pR5ePflRw')
df.loc[6545]=getTrackFeatures('371GoHoueVN53jY8MoWufq')
df.loc[6646]=getTrackFeatures('538hPQNtP190KcS3ZnWY5M')
df.loc[6689]=getTrackFeatures('5LGEfWqwgYL16tyXkrRC8J')
df.loc[6703]=getTrackFeatures('6OSmeoh8kXhCfz9WqvzDoJ')
df.loc[6713]=getTrackFeatures('2Hy7Snw1mgCk86DdDqJgV7')
df.loc[6729]=getTrackFeatures('6p6TjiJHc1kJQt5dXzkdrs')
df.loc[6873]=getTrackFeatures('0BAUo3Y2wyxtcArALvI1NI')
df.loc[6906]=getTrackFeatures('2DmBuFsXx1Pd5FHrMF8PD8')
df.loc[6925]=getTrackFeatures('7qFxSPDUDMQNGBqTvK4aEj')
df.loc[6940]=getTrackFeatures('3kqLybBT5LRSB9QCoR3ojK')
df.loc[6968]=getTrackFeatures('0nKZeSrqH9u31NLoTUsYBR')
df.loc[7005]=getTrackFeatures('7HhR1L61r3Z3PMWRRJSDHe')
df.loc[7079]=getTrackFeatures('3kZoay4ANo86ehb6s4RwS9')
df.loc[7099]=getTrackFeatures('2TNItTGFywUukXJdG6Qa25')
df.loc[7126]=getTrackFeatures('3r4U9UpHaFurMtq1oCZoAp')
df.loc[7346]=getTrackFeatures('5U1lQBg8zLFZCv9UzU6Noh')
df.loc[7480]=getTrackFeatures('7B42GdidcsaQdJCBiPtzSW')
df.loc[7561]=getTrackFeatures('0va4qYYci02sJcKnhv9tfC')
df.loc[7600]=getTrackFeatures('2b8QQqf8Vwb7bfz22ja8Fk')
df.loc[7608]=getTrackFeatures('69Rq57lrGf5syykFBNi3g4')
df.loc[7623]=getTrackFeatures('68vgtRHr7iZHpzGpon6Jlo')
df.loc[7634]=getTrackFeatures('7xXvPyOiFCsB5SSpio5g38')
df.loc[7668]=getTrackFeatures('5Lzumvaqcoa6eZLLQn0IW2')
df.loc[7672]=getTrackFeatures('507bYMYfbm6sUS9iEAaeSd')
df.loc[7732]=getTrackFeatures('6ihObRBTB8xdSH2mlERtOX')
df.loc[7736]=getTrackFeatures('4MMqj7HaLL8BphbBZnlsdq')
df.loc[7775]=getTrackFeatures('7sZCAHP2duHwr5M5K7lHsb')
df.loc[7779]=getTrackFeatures('2ISihV3VpUlngDWOlWDUI0')
df.loc[7853]=getTrackFeatures('0UNTBB02C1aB55Fu4H63qd')
df.loc[7908]=getTrackFeatures('6Nq5lE0CeOAAAqN4qhkbZU')
df.loc[8050]=getTrackFeatures('1HCPjfa02m5JwcO37LHfYb')
df.loc[8129]=getTrackFeatures('69j0KoPJuwpnbGWrfn7Yll')
df.loc[8145]=getTrackFeatures('7FrJV8tydWEv1Mxu2mIQrm')
df.loc[8223]=getTrackFeatures('2dCD8qq7yHZPZduA8FJazs')
df.loc[8248]=getTrackFeatures('5kDkWHFx0AEvJ1Pfm2i0ac')
df.loc[8286]=getTrackFeatures('0Uhnzk5zI46IRlQ04LNOtc')
df.loc[8301]=getTrackFeatures('0Uhnzk5zI46IRlQ04LNOtc')
df.loc[8327]=getTrackFeatures('6cZPfzVtkicrA3CyjqYKhu')
df.loc[8498]=getTrackFeatures('7x6ev0H9OAmvuDnyuItBZS')
df.loc[8632]=getTrackFeatures('43zr9kKkeiQrshvYuvNtfM')
df.loc[8633]=getTrackFeatures('55mJleti2WfWEFNFcBduhc')
df.loc[8635]=getTrackFeatures('1RKUoGiLEbcXN4GY4spQDx')
df.loc[8639]=getTrackFeatures('6CBRI05fAHw76JqDk2IRkK')
df.loc[8640]=getTrackFeatures('7qLQR9T8vL638MbPYCDAFk')
df.loc[8641]=getTrackFeatures('2zj1zujNxjBMBdfkm5uY2o')
df.loc[8642]=getTrackFeatures('7pbNNVcFePJV3Wani6zH8q')
df.loc[8644]=getTrackFeatures('4tgeQrPu5xkrnwErym2JsJ')
df.loc[8650]=getTrackFeatures('2eOuL8KesslTLQERQPu11D')
df.loc[8652]=getTrackFeatures('23qDMWnwf8p0pr5slljB6i')
df.loc[8653]=getTrackFeatures('4CmSNIT8dhpICN4Kp910Uz')
df.loc[8655]=getTrackFeatures('2seY1Vy7jidW4bxk98n2eh')
df.loc[8657]=getTrackFeatures('3yUcJwYu7fXAfqMj9krY6l')
df.loc[8660]=getTrackFeatures('0lyYVubxulG0uUPOVCQj0Z')
df.loc[8662]=getTrackFeatures('2648dyCgcTFNkJZXgGlHPv')
df.loc[8663]=getTrackFeatures('2TVSa07y6g3HRn4oLWv57M')
df.loc[8664]=getTrackFeatures('3uZAWnZVt2xL4X7RfNrCDk')
df.loc[8666]=getTrackFeatures('3xuobJ0EyP0Boajt7p8xWq')
df.loc[8667]=getTrackFeatures('0cHRuyr1YyvJXTCX5ZeqSO')
df.loc[8668]=getTrackFeatures('1s8MNZEUhMRlCJiFzHCkTY')
df.loc[8669]=getTrackFeatures('6Sy9BUbgFse0n0LPA5lwy5')
df.loc[8670]=getTrackFeatures('75b9JVzc4QFvPBO7jDRIVe')
df.loc[8671]=getTrackFeatures('75b9JVzc4QFvPBO7jDRIVe')
df.loc[8672]=getTrackFeatures('1AmupdZPhPTRkq82EXi83f')
df.loc[8674]=getTrackFeatures('34Vn9nKfztyLco9lJazy4j')
df.loc[8675]=getTrackFeatures('6nDUaxCt8veC1ZnfQFfKP5')
df.loc[8679]=getTrackFeatures('3Gf5nttwcX9aaSQXRWidEZ')
df.loc[8680]=getTrackFeatures('2fdoGgBd4CbyA4Tkw1Gyw2')
df.loc[8682]=getTrackFeatures('1Un1dsZkA9cfZMLOXOyUBc')
df.loc[8684]=getTrackFeatures('5rJRKtJzm7qacwYEjsKav6')
df.loc[8685]=getTrackFeatures('6Jvx6HCj3wxWWIs90YqWXj')
df.loc[8687]=getTrackFeatures('4PRB8Cs2ONx4VcVv0rFSwj')
df.loc[8691]=getTrackFeatures('154grZhOoDf2FDTnS2BpK6')
df.loc[8693]=getTrackFeatures('7fFCnttacpotkyhTOxKv14')
df.loc[8694]=getTrackFeatures('3hfMKCMfKcKctpjtH0TB76')
df.loc[8697]=getTrackFeatures('22WlwTY6Y41bG3hIOMF4VO')
df.loc[8700]=getTrackFeatures('2aGw77pZqli064JiPuXYRi')
df.loc[8703]=getTrackFeatures('3NJQ1KdbMLE1CEEVNjs9IO')
df.loc[8704]=getTrackFeatures('1PmJcgLwG1wRC5qMdjogRD')
df.loc[8707]=getTrackFeatures('1CP0DS790dxOfCXXp7WHDP')
df.loc[8708]=getTrackFeatures('0YDYMz9BP602o75O9jgr6l')
df.loc[8709]=getTrackFeatures('2Mpj1Ul5OFPyyP4wB62Rvi')
df.loc[8710]=getTrackFeatures('3NjAxulzcqR9riwzAnoN7p')
df.loc[8711]=getTrackFeatures('0YHI5hydPPth6J5UOzdLDX')
df.loc[8712]=getTrackFeatures('2rR9wUciUfXFrmf0h6rwMI')
df.loc[8713]=getTrackFeatures('2gSVKxPDww9Eep5rdvtdem')
df.loc[8715]=getTrackFeatures('04ccQGJ1i8gL7JoeCPiJbv')
df.loc[8716]=getTrackFeatures('0xikWgPgYN9BEes0ieZ8Co')
df.loc[8717]=getTrackFeatures('7fkbO9RJWW0zQAtXBu5ixb')
df.loc[8721]=getTrackFeatures('3oW6SWwGqiZSPTiAp7ZQoH')
df.loc[8723]=getTrackFeatures('6glsMWIMIxQ4BedzLqGVi4')
df.loc[8724]=getTrackFeatures('3OWsm3HE35S9x3fgY8tbgh')
df.loc[8725]=getTrackFeatures('5bCxcsT08RAtM4AvK9Gryi')
df.loc[8727]=getTrackFeatures('2MMZ1Llts0rTaU834P2Xct')
df.loc[8728]=getTrackFeatures('1sVv67vCTTKGBM5ZHYAcli')
df.loc[8732]=getTrackFeatures('4ohEI9gzIH5mSK6R6aUssT')
df.loc[8734]=getTrackFeatures('7FWuzwapuKHTEyDJIcQOw3')
df.loc[8738]=getTrackFeatures('0mu59f9C3CwjFdtmf0j80v')
df.loc[8741]=getTrackFeatures('0UaFVjNd7WxSb8hHHL6eql')
df.loc[8743]=getTrackFeatures('4IWCqcLq1AlO7hXDTQLZAe')
df.loc[8745]=getTrackFeatures('2EVNYzHWebyLvKxPX6Ozt8')
df.loc[8758]=getTrackFeatures('2AuJ7leOU5BKes4za742Jo')
df.loc[8883]=getTrackFeatures('1RUsaD5o5QjW6lDx50W7KP')
df.loc[8897]=getTrackFeatures('4WP1XJnAUkZfTtMdZTaN9k')
df.loc[8908]=getTrackFeatures('0lkyec6GEucNx8L9rJuLTK')
df.loc[8947]=getTrackFeatures('4x6kZErnzSn7TET3CmFvn5')
df.loc[8958]=getTrackFeatures('1ylplaqhSLPgMkHdNKJ1BT')
df.loc[9176]=getTrackFeatures('5q186AzhWeOEoFnApajHgR')
df.loc[9192]=getTrackFeatures('72MCDdQ1F4UfrOLl3ayLI6')

In [47]:
# Add an identifiable id to match with charts_no_duplicate and export for documentation
df['id'] = df.index + 1
df.to_csv('df.csv')